# Running Molecular Dynamics Simulations in OpenMM

## Molecular Dynamics Simulations

Now that we have our force field defined, we are ready to do a simulation! 
But, what does that mean and what kind of simulation will we be doing?
 We will use a simulation method called **molecular dynamics** (MD). 
 In molecular dynamics simulations, we simulate molecules in time by calculating the forces on atoms and updating their positions based on those forces. The output of a molecular dynamics simulation is a trajectory. 
 his is commonly written to a file on the computer where the calculation is running. 
 The trajectory file contains atomic positions for points in time. 
 The trajectory can be visualized to show the movement of a system through time (like watching a movie).

Force is equal to the negative gradient of potential energy. 
MD simulations use the potential energy function we discussed previously, and Newton's second law (the force on an object is equal to the object's mass times its acceleration) to calculate positions of atoms. 

$$ \vec{F} = - \nabla U $$

$$ \vec{F} = m \cdot \vec{a} = m \cdot \frac{d\vec{v}}{dt} = m \cdot \frac{d^{2}\vec{r}}{dt^{2}}$$

Looking at these two equations, we se that we we already know the mass for each atom. With our force field defined, we can calculate the force on each atom. The net force on each atom is the sum of forces from all other atoms in the system:

$$ \vec{F}_i = \sum_{j=1}^{n_{max}}{f_{j}}, (j \neq i) $$

This means that we should be able to calculate positions for each atom based on the force field function, the positions of other atoms in the system, and mass of the molecule. 

$$ \vec{a}_{i}(t_{0}) = \frac{1}{m}\vec{F}_{i}(t_{0}) = -\frac{1}{m}_{i}(\nabla U)_{i} $$

We know that we can calculate positions ($\vec{x}$) based on previous positions and acceleration and an amount of time ($\Delta t$)

$$\vec{x}_i (t_{0} + \Delta t) = \vec{x}_i (t_{0}) + \vec{v}_{i} (t_{0}) \Delta t + \frac{1}{2} \vec{a}_{i}(t_{0}) \Delta t^{2} $$

This is the general approach to molecular dynamics. You have some atoms which have initial positions and velocities when you start your simulation. You can use these positions to calculate the potential energy and the forces on each atom (because force is the negative gradient of potential energy). With the velocities of the atoms, you can calculate new positions. This process is repeated over and over again, and each iteration is typically called a **timestep**. 

The typical time step size ($\Delta t$) represents 1 femtosecond of time and is based on X-H bond vibration frequencies. 
A timestep that is too small will lead to an inefficient simulation, while a time step that is too large will be inaccurate or cause your simulation to fail. Often, we will constrain hydrogen in a simulation and can use a longer timestep of 2 femtoseconds. 
MD simulations assumes that the [Erogodic hypothesis](https://en.wikipedia.org/wiki/Ergodic_hypothesis) is true, meaning that the time average calculated from the simulation is equal to the ensemble average. 
For this to be true, the simulation has to sample many conformations. 
Most simulations have millions to trillions of time steps. 



## Steps for running a simulation

Our steps for running a simulation will be very similar to the steps we used last week for our Monte Carlo simulation.
However, because our system is more complicated, we add a minimization step before the equilibration step. 
This is typical for a molecular dynamics simulation.

1. **Initialization** -- setting up imports, variables, and other system parameters to prepare for the simulation
2. **Minimization** -- calculating the energies of moving molecules slightly to obtain a local "minimum" in energy and eliminate “bad” interatomic contacts
3. **Equilibration** -- a brief MD simulation that prepares our system to our target temperature and desired equilibrium values
4. **Production** -- run the equilibration
5. **Analysis** -- after you have collected your data from the production run, you must analyze the trajectory to draw conclusions

## Our First Simulations
We will now use [OpenMM](https://openmm.org/documentation) to do a molecular dynamics simulation of the ethane and butane molecules we prepared. It’s important to note at this point that molecular dynamics simulations can be performed using a number of softwares. However, we will be running a simulation with a program called OpenMM. OpenMM has the advantage of being scriptable with Python.

OpenMM is also highly *object oriented*. When using OpenMM, everything about a simulation is represented through an object. 
This makes it relate well to our discussion of OOP in this class!

---

### Initialization
Once you have OpenMM, we can use it to simulate our molecules. Start in your notebook with imports. Here are the python libraries you will need to run simulations with OpenMM.

We are also importing a library called [NGLView](http://nglviewer.org/nglview/latest/) which can be used to make molecular visualizations in Jupyter notebooks.

In [ ]:
import openmm as mm
from openmm import app, unit

import nglview as nv

This first cell is just used to visualize our starting ethane molecule.

In [ ]:
starting_system_file = "ethane/ethane.pdb"

view = nv.show_file(starting_system_file)
# Clear default representation
view.clear_representations()

# Add a new representation
view.add_representation("ball+stick")

view

First, we need to read in our structure and our force field. We have to tell the simulation our initial coordinates and the force field we will use. To do this, we use the PDB file we have and the force field file.
The forcefield parameters are stored in an XML file in the `ethane` folder.

In [ ]:
pdb = app.PDBFile(starting_system_file)
forcefield = app.ForceField('ethane/ethane.gaff2.xml')

#### QUESTION
What are we doing with these two commands above?
*(Hint: let's check out some [documentation](http://docs.openmm.org/latest/userguide/application/02_running_sims.html))*

#### RESPONSE
Your response here :D

#### QUESTION
What lies in the contents of the XML file being read in `forcefield`?

#### RESPONSE
Your response here :D

Next, we set up the system for our MD simulation. With the following command, we use the `pdb` variable to create a system. The other arguments to the function say that we are not using a cut-off and that we want to constrain bonds with hydrogens (this allows us to use a larger timestep).

In [ ]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

Our simulation will be in vacuum at a temperature of 298.15 K. The Langevin integrator is what is called a stochastic integrator. This means that it mimics jostling of air or solvent through random forces. We are using a 5.0 picosecond coupling constant, which is something which controls how often the integrator adds jostling motions.

In [ ]:
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

#### QUESTION
What is the `integrator` applying to our simulation?

#### RESPONSE
Your response here :D

Finally, we initialize the simulation by adding all of the pieces we have prepared:

In [ ]:
# run on a GPU if available.
try:
    platform = mm.Platform.getPlatformByName('CUDA')
except:
    platform = mm.Platform.getPlatformByName('Reference')

# create simulation object.
simulation = app.Simulation(pdb.topology, system, integrator, platform)

# set the initial positions of the atoms.
simulation.context.setPositions(pdb.positions)

---

### Minimization
Now, we start calculating energies. First we do an energy minimization. We start in this code block by printing the energy before minimization, doing 100 steps of an energy minimization, then printing the new energy. You should see that the energy decreases:

In [ ]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st.getPotentialEnergy()}")

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st.getPotentialEnergy()}")

## Exercise 

Use NGLView to visualize the minimized structure.
The next cell saves the minimized structure as a molecular file format called a PDB.

In [ ]:
# Save the minimized structure as a pdb file.

minimized_structure = "ethane/minimized.pdb"
f = open(minimized_structure, "w+")
app.PDBFile.writeFile(simulation.topology, simulation.context.getState(getPositions=True).getPositions(), f)
f.close()

In [ ]:
## Your code here
## Use NGLView to visualize the minimized structure.

view = nv.show_file(minimized_structure)
# Clear default representation
view.clear_representations()

# Add a new representation
view.add_representation("ball+stick")

view

#### QUESTION
What happened during the minimization? 
How is the minimized structure different than the starting structure?
Why do you think this happens?

#### RESPONSE
Your response here :D

#### QUESTION
What is an energy minimization? Why do we want to perform a local energy minimization at the start of our simulation?

#### RESPONSE
Your response here :D

---

### Equilibration
Next, we run an equilibration. The purpose of this equilibration is to get our system to our target temperature and to get the system equilibrated and ready for our production run.

Comment each line of code in the cell below.

In [ ]:
# Comment each line of code in this cell.

from sys import stdout

print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.reporters.append(app.StateDataReporter("ethane/equilibration.csv", 100, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(5000)

The first command in this section sets up what information OpenMM will report to us as the simulation runs. We are asking for the potential energy, and temperature every 100 timesteps. By putting `stdout` as the first argument, we are telling the program to just print the information. Note that `stdout` comes from the built in Python module `sys`. If we wanted the information in a file instead, you would put the file name.

The second command sets the initial velocities of the system to a temperature equivalent of 150 K. Then, we integrate for 2,500 steps to allow the system to equilibrate.

After we perform an equilibration, we will make some plots to visualize what happened. This is similar to what we did last week.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
equilibration_data = pd.read_csv("ethane/equilibration.csv")
equilibration_data.head()

In [ ]:
plt.plot('#"Step"',"Potential Energy (kJ/mole)", data=equilibration_data)

In [ ]:
plt.plot('#"Step"',"Temperature (K)", data=equilibration_data)

We can also check the average temperature to see if we are near our target.
When we are simulating such a small system such as a single molecule, it will be typical to see a somewhat large variation in temperature.
Here, we'll just visually check that the temperature is near 300 K.

In [ ]:
equilibration_data["Temperature (K)"].mean()

---

### Production
This next block of code is a longer block of simulation called the ‘production run’. We’ve also added a timer to this code so we can see how long it took our simulation to run.

In [ ]:
import time as time

print('Running Production...')

# Begin timer
tinit=time.time()

# Clear simulation reporters
simulation.reporters.clear()

# Reinitialize simulation reporters. We do this because we want different information printed from the production run than the equilibration run.
# output basic simulation information below every 250000 steps - (which is equal to 2 fs(250,000) = 500,000 fs = 500 ps)
simulation.reporters.append(app.StateDataReporter(stdout, 250000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))

simulation.reporters.append(app.StateDataReporter("ethane/production.csv", 10000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))

# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps - 0.2 ps
simulation.reporters.append(app.DCDReporter('ethane/trajectory.dcd', 100))

# run the simulation for 2.0x10^6 steps - 4 ns
simulation.step(2000000)

# End timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

After executing this cell in your notebook, you should see an output which gives you the step number, simulation time, potential energy, temperature, and “speed” for steps in the simulation. The spacing of theses is set in the `simulation.reporters` step where we indicated we wanted information printed every 250,000 timesteps.

The “speed” is reported in “ns/day” or “nanoseconds/day”. This is a commonly used unit to report how quickly simulations run. It tells you how much simulation time in nanoseconds would pass for 24 hours of computation time. For example, if a simulation is running at 2 ns/day, enough timesteps would be calculated in one day to make 2 nanoseconds of simulation time. If we were using our 2 fs timestep, this would mean that the computer calculated 1,000,000 timesteps over 24 hours.

In [ ]:
production_data = pd.read_csv("ethane/production.csv")
production_data.head()

In [ ]:
plt.plot('#"Step"',"Potential Energy (kJ/mole)", data=production_data)

---

## Your Turn
Make a copy of the code you wrote to run your ethane simulation and modify it to:
1. Read in the files `butane.gaff2.xml` and `butane.pdb`
2. Carry out a 10 ps MD simulation to bring the butane molecule to an equilibrium temperature of 298 K in which output is printed every 0.5 ps (Leave the minimization portion beforehand unchanged.)
3. Carry out a 40 ns MD simulation at 298 K in which output is printed every 1 ns and structures are (still) saved every 0.2 ps into a file called `butane_sim.dcd`.

In [ ]:
### your code here!!!

---

## Analysis
Now that we’ve performed our computer experiment, it is time to analyze the data we have collected. The main type of data you have collected through this simulation is information on atom positions, or the system trajectory.

As part of our production simulation, we set up a reporter to record atomic positions. The code below shows that code from your previous script (this is not executable, it's just to show you)

```python
simulation.reporters.append(app.DCDReporter('trajectory.dcd', 100))
```

This reporter saved the atomic positions for us every 100 timesteps in a file called `ethane_sim.dcd`. The DCD file format is a binary file (instead of being a text file), so you cannot open it and look at it. However, we will be using certain libraries to analyze and view the file’s contents. If you’ve run your simulation, you should have the file `ethane_sim.dcd` in the same folder as your Jupyter notebook.

We are going to use a Python package called [MDAnalysis](https://docs.mdanalysis.org/stable/index.html) to analyze our trajectories.
MDAnalysis is commonly used to analyze molecular dynamics simulations, though there are many other choices.
MDAnalysis, like OpenMM, is very highly object oriented.

When you start analyzing a simulation using MDAnalysis, you use output from your simulation to create an object called an MDAnalysis "universe".
A "universe" contains all information about the system in a molecular dynamics simulation.

In [ ]:
import MDAnalysis as mda

# The first argument is the file with the topology (or atom connectivity information)
# the second file is the trajectory file.
u = mda.Universe("ethane/ethane.pdb", "ethane/trajectory.dcd")

The command above reads all of the atomic positions from `ethane_sim.dcd` and keeps track of atom connectivity (topology) which was given in the PDB file. Next, visualize the trajectory using nglview. Nglview has a special function `show_mdanalysis` that we can use with our trajectory because it was in a specific format from the MDAnalysis library.

You can watch a movie of the trajectory by clicking the "play" (arrow) button in the lower left.

In [ ]:
view = nv.show_mdanalysis(u)
view

This should show you something that looks sort of like a movie of your ethane molecule. These are the atomic positions calculated by OpenMM during the molecular dynamics run. We can now analyze the positions to find out some things about our molecule.

We will use another OpenMM command to pull out our bonds and atoms for analysis

### Analyzing the C-C bond length
Let’s look at what C-C bond lengths our ethane molecule had during the simulation. Before we can measure the bond lengths, we have to decide which atoms from our molecule define the bond distance.

First, we will see the information stored about each atom in MDAnalysis. You'll notice from this for loop that each atom is represented by an atom object.

In [ ]:
# reminder - u is our MDAnalysis "universe" - an object containing
# information about the system we simulated.
for atom in u.atoms:
    print(atom)

We will use information to select our carbon atoms. We will use the atom names for our selection.

In [ ]:
carbon_1 = u.select_atoms("name C1")
carbon_2 = u.select_atoms("name C2")

print(carbon_1)
print(carbon_2)

Next, we will use the `calc_bonds` function in `mda.lib.distances` to compute the distance between our selected atoms.
We measure the distance between these two atoms for each simulation frame.
We create a list called `bond_length`, and append the measurement for each simulation frame to this list.

In [ ]:
bond_length = []

for ts in u.trajectory:
    distance = mda.lib.distances.calc_bonds(carbon_1.positions, carbon_2.positions)
    bond_length.append(distance[0])

We now have the measurement for the C-C bond length for each recorded timestep of the trajectory saved in the array `bond_length`. One way we can examine this data is by plotting it as a histogram using the Python library `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

bondcounts, binedges, otherstuff = plt.hist(bond_length, bins=120)
plt.title('C-C bond length histogram')
plt.xlabel('Bond length (angstrom)')
plt.ylabel('Counts')

#### QUESTION
What does this plot tell you about the bond length? Compare each plot to the potential energy plot you made for that interaction in the previous notebook.

#### RESPONSE
Your response here :D

#### Exercise

Fill in the cells belwo to measure the angle between `H11`, `C1`, and `C2` in your simulation using MDAnalysis.
For this measurement, you should use `mda.lib.distances.calc_angles`.

In [ ]:
hydrogen_11 = # your code to select H11 here.
print(hydrogen_11)

In [ ]:
HCC_angle = []

for ts in u.trajectory:
    # Your code to measure the angles here.

In [ ]:
# Create a histogram of angles here.

#### Exercise

Fill in the cells belwo to measure the torision (dihedral) between `H11`, `C1`, `C2`, and `H12` in your simulation using MDAnalysis.
For this measurement, you should use `mda.lib.distances.calc_dihedrals`.

In [ ]:
hydrogen_21 = # select H21 here
print(hydrogen_21)

In [ ]:
# Your code to measure the torsions here

In [ ]:
# Visualize the histogram here.

## Your Turn
Repeat the analysis of C-C bond length, a bond angle of your choice (can be C-C-H or C-C-C), and a dihedral (use the dihedral angle of the carbons)
for your butane simulation.


#### QUESTION
What does each plot tell you? Compare each plot to the potential energy plot you made for that interaction in the previous notebook.

#### RESPONSE
Your response here :D

Credit for this exercise goes to [MolSSI](https://education.molssi.org/mm-tools/02-md-alkanes/index.html).